In [1]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
import joblib
import numpy as np

In [2]:
from abc import ABC, abstractmethod

class ModelBase(ABC):
    def __init__(self):
        self.model = None
        self.features = None
    
    @abstractmethod
    def train(self, dataset, features):
        """
        Train prediction model
        Dataset: dataset
        Traning features: features
        """
        pass

    @abstractmethod
    def load(self, modelPath):
        """
        Load saved model
        """
        pass
    
    @abstractmethod
    def save(sefl, path):
        """
        Save model
        """
        pass


    @abstractmethod
    def predict(self, dataToPredict):
        """
        Predict candle price
        """
        pass

In [3]:
class LSTMModel(ModelBase):
    def __init__(self):
        self.T = 60
        self.x_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        super().__init__()
    
    def train(self, dataset, features):
        # dataset includes the following columns: Open, High, Low, Close, Adj Close, Volume according to the order
        print("Training LSTM model")
        x_train = []
        # y_train includes the following columns: Open, High, Low, Close according to the order
        y_train = []

        x_scaled = self.x_scaler.fit_transform(dataset)
        y_scaled = self.y_scaler.fit_transform(dataset[:, 0:4])
        
        for i in range(self.T, len(dataset)):
            x_train.append(x_scaled[i-self.T:i])
            y_train.append(y_scaled[i])

        x_train, y_train = np.array(x_train), np.array(y_train)
        
        self.model = self.__create_model(x_train)
        self.model.fit(x_train, y_train, epochs=10, batch_size=32)
    
    def load(self, path):
        # Load the model
        self.model = load_model(path + "/model.h5")
        # Load the scaler
        self.scaler = joblib.load(path + "/scaler.save")
        print("Model and scaler loaded from", path)
    
    def save(self, path):
        # Save the model
        self.model.save(path + "/model.h5")
        # Save the scaler
        joblib.dump(self.scaler, path + "/scaler.save")
        print(f"Model and scaler saved to {path}")

    def predict(self, dataToPredict):
        x_test = []
        dataToPredict = self.x_scaler.transform(dataToPredict)
        x_test.append(dataToPredict[-self.T:])
        x_test = np.array(x_test)
        prediction = self.model.predict(x_test)
        prediction = self.y_scaler.inverse_transform(prediction)
        return prediction

    def __create_model(self, x_train):
        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
        model.add(Dropout(0.2))
        model.add(LSTM(units=50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(units=50))
        model.add(Dropout(0.2))
        model.add(Dense(units=4))
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

In [4]:
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

from datetime import datetime

def load_data():
    df = pdr.get_data_yahoo('AAPL', start='2016-01-01', end=datetime.now())
    dataset = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].values
    return dataset

def test():
    dataset = load_data()
    model = LSTMModel()
    model.train(dataset, ['Open', 'High', 'Low', 'Close'])
    model.save("./trained")

    prediction = model.predict(dataset[-60:])
    print("Prediction:", prediction)

def test_from_load():
    dataset = load_data()
    model = LSTMModel()
    model.load("./trained")
    prediction = model.predict(dataset[-60:])
    print("Prediction:", prediction)

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [5]:

model = LSTMModel()
model.train(load_data(), ['Open', 'High', 'Low', 'Close'])

[*********************100%%**********************]  1 of 1 completed

Training LSTM model
Epoch 1/10



d:\Programming\Python\Stock Prediction Dashboard\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 0.2599
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0274
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0240
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0222
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0192
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0200
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0183
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0188
Epoch 9/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0195
Epoch 10/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.0149


In [10]:
x_test = load_data()[-61:]
# drop the last row
x_test = x_test[:-1]


prediction = model.predict(x_test)
prediction = prediction[0]

[*********************100%%**********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [11]:
prediction

array([207.97746, 210.33755, 206.8964 , 208.63216], dtype=float32)

In [9]:
df = pdr.get_data_yahoo('AAPL', start='2016-01-01', end=datetime.now())
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,25.652500,26.342501,25.500000,26.337500,23.914484,270597600
2016-01-05,26.437500,26.462500,25.602501,25.677500,23.315199,223164000
2016-01-06,25.139999,25.592501,24.967501,25.174999,22.858938,273829600
2016-01-07,24.670000,25.032499,24.107500,24.112499,21.894176,324377600
2016-01-08,24.637501,24.777500,24.190001,24.240000,22.009949,283192000
...,...,...,...,...,...,...
2024-06-26,211.500000,214.860001,210.639999,213.250000,213.250000,66213200
2024-06-27,214.690002,215.740005,212.350006,214.100006,214.100006,49772700
2024-06-28,215.770004,216.070007,210.300003,210.619995,210.619995,82542700
